In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip /content/drive/MyDrive/izcc-ailecture-1.zip
!mkdir train_data
!mv /content/train_Brain_Tumor/train_Brain_Tumor train_data
!mv /content/train_Healthy/train_Healthy train_data
!mv /content/final_test_dataset/final_test_dataset test

Archive:  /content/drive/MyDrive/izcc-ailecture-1.zip
  inflating: final_test_dataset/final_test_dataset/100_2.jpg  
  inflating: final_test_dataset/final_test_dataset/101_2.jpg  
  inflating: final_test_dataset/final_test_dataset/102_2.jpg  
  inflating: final_test_dataset/final_test_dataset/103_2.jpg  
  inflating: final_test_dataset/final_test_dataset/104_2.jpg  
  inflating: final_test_dataset/final_test_dataset/105_2.jpg  
  inflating: final_test_dataset/final_test_dataset/106_2.jpg  
  inflating: final_test_dataset/final_test_dataset/107_2.jpg  
  inflating: final_test_dataset/final_test_dataset/108_2.jpg  
  inflating: final_test_dataset/final_test_dataset/109_2.jpg  
  inflating: final_test_dataset/final_test_dataset/10_2.jpg  
  inflating: final_test_dataset/final_test_dataset/110_2.jpg  
  inflating: final_test_dataset/final_test_dataset/111_2.jpg  
  inflating: final_test_dataset/final_test_dataset/112_2.jpg  
  inflating: final_test_dataset/final_test_dataset/113_2.jpg  
  

In [6]:
import os
import shutil

def create_train_validation_folders(source_folder, train_folder, validation_folder, split_size):
    # 創建訓練集和驗證集的目錄，如果它們不存在的話
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(validation_folder, exist_ok=True)

    # 獲取源文件夾中的子文件夾列表
    subfolders = [folder for folder in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, folder))]

    # 對每個子文件夾執行操作
    for subfolder in subfolders:
        print(subfolder)
        # 構建子文件夾的完整路徑
        subfolder_path = os.path.join(source_folder, subfolder)
        train_subfolder_path = os.path.join(train_folder, subfolder)
        validation_subfolder_path = os.path.join(validation_folder, subfolder)

        # 創建訓練集和驗證集中的子文件夾，如果它們不存在的話
        os.makedirs(train_subfolder_path, exist_ok=True)
        os.makedirs(validation_subfolder_path, exist_ok=True)

        # 獲取子文件夾中的文件列表
        files = os.listdir(subfolder_path)

        # 將文件按照指定的split_size複製到訓練集中
        for file in files[:split_size]:
            source_file = os.path.join(subfolder_path, file)
            destination_file = os.path.join(train_subfolder_path, file)
            shutil.copyfile(source_file, destination_file)

        # 將文件從split_size到1800的部分複製到驗證集中
        for file in files[split_size:1800]:
            source_file = os.path.join(subfolder_path, file)
            destination_file = os.path.join(validation_subfolder_path, file)
            shutil.copyfile(source_file, destination_file)


In [7]:
source_folder = "/content/train_data"
train_folder = "/content/train valid/train"
validation_folder = "/content/train valid/validation"
split_size = 1500

create_train_validation_folders(source_folder, train_folder, validation_folder, split_size)

train_Brain_Tumor
train_Healthy


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = "/content/train valid/train"
validation_data_dir = "/content/train valid/validation"

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    classes=['train_Healthy', 'train_Brain_Tumor'],
    shuffle=True ,
    color_mode="grayscale"
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    classes=['train_Healthy', 'train_Brain_Tumor'],
    shuffle=False,
    color_mode="grayscale"
)

Found 3000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [13]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

model = keras.Sequential([
    layers.Input(shape=(150, 150, 1)),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.2),
    layers.Rescaling(1./255), #normaliztion
    layers.Conv2D(filters=32, kernel_size=3, activation="relu"),
    layers.MaxPooling2D(pool_size=2),
    layers.Conv2D(filters=64, kernel_size=3, activation="relu"),
    layers.MaxPooling2D(pool_size=2),
    layers.Conv2D(filters=128, kernel_size=3, activation="relu"),
    layers.MaxPooling2D(pool_size=2),
    layers.Dropout(0.2),
    layers.Conv2D(filters=256, kernel_size=3, activation="relu"),
    layers.MaxPooling2D(pool_size=2),
    layers.Dropout(0.25),
    layers.Conv2D(filters=512, kernel_size=3, activation="relu"),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

In [14]:
model.compile(loss="binary_crossentropy",
       optimizer="rmsprop",
       metrics=["accuracy"])

In [15]:
modelsaving = "/content/drive/MyDrive/best_model.h5"

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=modelsaving,
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=callbacks)

Epoch 1/25
94/94 [==============================] - 196s 2s/step - loss: 0.6937 - accuracy: 0.5060 - val_loss: 0.6951 - val_accuracy: 0.5000
Epoch 2/25
94/94 [==============================] - 203s 2s/step - loss: 0.6938 - accuracy: 0.5003 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 3/25
94/94 [==============================] - 203s 2s/step - loss: 0.6938 - accuracy: 0.4963 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/25
94/94 [==============================] - 191s 2s/step - loss: 0.6936 - accuracy: 0.4960 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 5/25
94/94 [==============================] - 203s 2s/step - loss: 0.6936 - accuracy: 0.5020 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 6/25
94/94 [==============================] - 203s 2s/step - loss: 0.6937 - accuracy: 0.4957 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/25
94/94 [==============================] - 193s 2s/step - loss: 0.6936 - accuracy: 0.4983 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/25
94

In [17]:
import pandas as pd
test_df="/content/train_data"

test_df = test_df.values.reshape((28000, 28, 28, 1))
test_df = test_df.astype("float32") / 255

predict=model.predict(test_df)

AttributeError: ignored